# 이 코드는 우분투 3090에서 사용할수 있게 만들어졌습니다

In [ ]:
# 알파카 모델 파인 튜닝해보기

In [1]:
!nvidia-smi

Fri May 19 21:29:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:65:00.0 Off |                  N/A |
|  0%   28C    P8    19W / 350W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 설치

In [2]:
""!git clone -q https://github.com/kairess/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08""

/home/perfact/fine_tune/alpaca-lora


In [ ]:
# 에러 디버깅 
# 이거 한다고 조금 죽을뻔함
## bitsandbytes 에러 -> cuda 인식이 잘 안되면 이런 오류가 발생하는듯 강제로 cpu에다가 cuda 동작하게 변경함
# cudnn이 조금 중요한듯 합니다.
"""
git clone https://github.com/timdettmers/bitsandbytes.git
cd bitsandbytes

CUDA_VERSION=118 make cuda11x # 본인에 쿠다 버전에 맞게 nvcc -V로 확인하기 
python setup.py install


설치후 bitsandbytes 파일 찾기
import inspect
import bitsandbytes

print(inspect.getfile(bitsandbytes))
# 해당 파일 위치가 나올텐데 거기서 
cp libbitsandbytes_cuda118.so libbitsandbytes_cpu.so 이런식으로 cpu 버전을 cuda118 버전으로 바꿔줍니다
## cuda version 확인은 nvcc -V로 해서 똑 같이 셋팅 해주셔야 합니다.
"""

In [ ]:
# 데이터 만들기

In [8]:
import pandas as pd
data = pd.read_csv('train_data3.csv') # 데이터 소스 케글 iab category
data['query'] = data['title'] + ' ' + data['content'] # title과 본문 축약 내용이 들어가 있습니다. 확인해주세요

data = data[~data['query'].isna()]
data = data[~data['label'].isna()] # 빈 데이터가 있어서 해당 작업 필요합니다.

In [ ]:
# json 포맷으로 저장
# instruction : 작업할 지시 항목
# input : 카테고리 하고 싶은 데이터
# output : 라벨 데이터 

In [ ]:

import json
output = []
for _, row in data.iterrows():
    entry = {
        'instruction': 'iab category of this data',
        'input': row['query'],
        'output': row['label']
    }
    output.append(entry)

# Convert the JSON data to a string
json_data = json.dumps(output, indent=2)

# Print the JSON data
print(json_data)

,query,label
0,5 Looks That Put Burgundy Boots On Our Must-Ha...,Womens\ Casual\ Wear
1,How to Make the Most of Your 401K According to...,Personal\ Investing
2,Mindful Eating “Thou shouldst eat to live; not...,Mens\ Health
3,Early life adversity and high levels of FKBP5 ...,Hormonal\ Disorders
4,Top 10 Strongest (Highest ATK) Monsters in Yu-...,Card\ Games
...,...,...
9162,Koji from It Might Get loud Production brings ...,Audio\ Production
9163,"Go Ahead, Have a Heart Tonight is it! The year...",Mens\ Health
9164,The Mobile Web Development Buffet In modern Mo...,Web\ Hosting
9165,Stirring the Plot: New Cookbooks Offer Recipes...,Cookbooks


In [7]:
"""
# finetuning 방법
# alphca_lora 폴더에서 

!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'output.json' \
    --output_dir './output' \
    --num_epochs 1 \
    --learning_rate 5e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'alpaca'
    """

python: can't open file 'finetune.py': [Errno 2] No such file or directory
